<a href="https://colab.research.google.com/github/bdurk/First_ever/blob/main/Data_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data wrangling with pandas
DS 1002: Programming for Data Science



### PREREQUISITES
- variables
- data types
- operators
- list comprehensions (not essential)
- numpy arrays (not essential)
- earlier pandas notebooks


### SOURCES
- ten minutes to pandas  
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html


- lambda (anonymous) functions  
https://realpython.com/python-lambda/#anonymous-functions


- pivot_table()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html


- concat()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html


- merge()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html


- get_dummies()  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html



### OBJECTIVES
- Introduce more advanced pandas dataframe operations for data munging



### CONCEPTS

- DataFrame
- `apply()`
- aggregation using split-apply-combine
- `pivot_table()`
- `groupby()`
- `concat()`
- merging/joining dataframes with `merge()`, `concat()`
- reshaping data
- dummy coding categorical data


---

## Additional Pandas DataFrame Functionality

In earlier pandas notebooks we covered *data frames* (creating, modifying, subsetting, etc) and *cleaning*.  
These notes will demonstrate further methods for data munging and analysis.

In [1]:
import numpy as np
import pandas as pd

#### Load Iris

In [4]:
import seaborn as sns
iris = sns.load_dataset('iris')

In [ ]:
iris.head(2)

In [6]:
# return the size of the object in bytes
import sys
sys.getsizeof(iris)

14744

## Apply Lambda Functions with `.apply()`

Apply a transformation to each record. Uses a `lambda` function.

In [ ]:
(lambda x: x**2) (3)

# high_ord_func = lambda x, func: x + func(x)
# high_ord_func(23, lambda x: x * x)


In [ ]:
iris['sepal_len_sq'] = iris.sepal_length.apply(lambda x: x**2)
iris.head(5)

Transformation involving multiple columns. Uses `axis=1` to access columns.  
Compute average of `sepal_length`, `sepal_width`:

In [ ]:
iris['sepal_len_wid_avg'] = iris[['sepal_length','sepal_width']].apply(lambda x: (x.sepal_length+x.sepal_width)/2, axis=1)
iris.head()

### TRY FOR YOURSELF (UNGRADED EXERCISES)

1) Use `apply()` to append a new column that is the minimum of (petal_length, petal_width)

Print the head, tail of the new dataframe to check things look correct.

In [17]:

iris['petal_length_and_width_min']=iris[['petal_length','petal_width']].apply(lambda x: min(x), axis=1)

iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_and_width_min
145,6.7,3.0,5.2,2.3,virginica,2.3
146,6.3,2.5,5.0,1.9,virginica,1.9
147,6.5,3.0,5.2,2.0,virginica,2.0
148,6.2,3.4,5.4,2.3,virginica,2.3
149,5.9,3.0,5.1,1.8,virginica,1.8


## Aggregation

Involves one or more of:

- splitting the data into groups
- applying a function to each group
- combining results

### `.groupby()`

Compute mean of each column, grouped (separately) by species

In [ ]:
iris.groupby("species").mean()

### `pd.pivot_table()`

Apply a function `aggfunc` to selected values grouped by columns

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [18]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

print(df)
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                       columns=['C'], aggfunc="sum")
table

     A    B      C  D  E
0  foo  one  small  1  2
1  foo  one  large  2  4
2  foo  one  large  2  5
3  foo  two  small  3  5
4  foo  two  small  3  6
5  bar  one  large  4  6
6  bar  one  small  5  8
7  bar  two  small  6  9
8  bar  two  large  7  9


C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

Compute mean sepal length for each species:

In [19]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)

species,setosa,versicolor,virginica
sepal_length,5.006,5.936,6.588


### TRY FOR YOURSELF (UNGRADED EXERCISES)

2) Use a pivot table to compute the following statistics on sepal_width and petal_width grouped by species:

- median  
- mean

These can be computed together in a single call to `pivot_table()`.

In [25]:
pd.pivot_table(iris, values=['sepal_width', 'petal_width'], columns=['species'], aggfunc={np.median, np.mean})


species             setosa  versicolor  virginica
petal_width mean     0.246       1.326      2.026
            median   0.200       1.300      2.000
sepal_width mean     3.428       2.770      2.974
            median   3.400       2.800      3.000

## Stacking and Unstacking

Similar to pivoting, but requires -- and takes advantage of -- indexes.

In [26]:
iris_w_idx = iris.copy()

# Give the original index a name
iris_w_idx.index.name = 'obs_id'

# Create a multi-index, using `species` as part of the key.
iris_w_idx = iris_w_idx.reset_index().set_index(['species','obs_id'])

In [ ]:
iris_w_idx

## `.unstack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html)

In [29]:
iris_wide = iris_w_idx.sepal_length.unstack(fill_value=0).T

In [ ]:
iris_wide

## `.stack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html)

In [31]:
iris_wide.T.stack().to_frame('sepal_length')

sepal_length
species   obs_id              
setosa    0                5.1
          1                4.9
          2                4.7
          3                4.6
          4                5.0
...                        ...
virginica 145              6.7
          146              6.3
          147              6.5
          148              6.2
          149              5.9

[450 rows x 1 columns]

## Combining DataFrames


### `pd.concat()`  

Concatenate pandas objects along an axis

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

Create two dfs and vertically stack them

In [34]:
df1 = pd.DataFrame(np.random.randn(3, 4))
df2 = pd.DataFrame(np.random.randn(3, 4))

print(df1)
print('-'*45)
print(df2)

df3 = pd.concat([df1, df2], axis=0)

print('-'*45)
print(df3)

          0         1         2         3
0  0.346446  0.629189 -1.839233 -1.066307
1  0.874937 -0.364392 -1.225955 -1.311697
2  0.485422  0.225795 -1.237246  0.025711
---------------------------------------------
          0         1         2         3
0  0.753824 -0.621870  1.040556 -0.211084
1 -0.070569 -1.641505 -0.339292 -1.248451
2  0.044905  0.964942  0.051285  0.455448
---------------------------------------------
          0         1         2         3
0  0.346446  0.629189 -1.839233 -1.066307
1  0.874937 -0.364392 -1.225955 -1.311697
2  0.485422  0.225795 -1.237246  0.025711
0  0.753824 -0.621870  1.040556 -0.211084
1 -0.070569 -1.641505 -0.339292 -1.248451
2  0.044905  0.964942  0.051285  0.455448


**Concat columns**  
This assumes that the indexes represent IDs of specific things or events

In [33]:
df4 = pd.concat([df1,df2], axis = 1, keys = ['foo', 'bar'])

df4

foo                                     bar                      \
          0         1         2         3         0         1         2   
0 -1.771045  2.115320  1.120627  0.256260 -2.468580  0.485833 -0.683020   
1 -0.296859  1.669234  0.743239  0.133385 -0.807002 -2.322474  0.388503   
2 -0.969645 -1.544764 -0.395162  0.805818 -0.917175  0.222321 -1.015500   

             
          3  
0  0.444508  
1  0.727192  
2  0.936912

In [35]:
df4.foo

,0,1,2,3
0,-1.771045,2.115320,1.120627,0.256260
1,-0.296859,1.669234,0.743239,0.133385
2,-0.969645,-1.544764,-0.395162,0.805818


In [ ]:
df4.bar

### `merge()`

SQL-style joining of tables (DataFrames)

Important parameters include:

- `how` : type of merge {'left', 'right', 'outer', 'inner', 'cross'}, default ‘inner’
- `on`  : names to join on
        
[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Create two tables, `left` and `right`. Then right join them on `key`.  
Right join means include all records from table on right.  
The `key` is used for matching up the records.

In [ ]:
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

Notice the NaN inserted into the record with key=asher, since the left table didn't contain the key.

**Matching column names**  
In this next example, the value columns have the same name: *val*.  Notice what happens to the column names.

In [ ]:
left = pd.DataFrame({"key": ["jamie", "bill"], "val": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "val": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

## `.join()`

An SQL-like joiner, but this one takes advantage of indexes.

Give our dataframes indexes and distinctive columns names.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)

In [ ]:
left2 = left.set_index('key').rename(columns={'val':'val_1'})
right2 = right.set_index('key').rename(columns={'val':'val_2'})

In [ ]:
left2

In [ ]:
right2

In [ ]:
right2.join(left2) # defaults to first df (right2 in this case)

In [ ]:
right2.join(left2, how = 'inner')

## Summary

* Use **join** if you have shared indexes
* Use **merge** if you do not have shared indexes
* Use **concat** to combine based on shared indexes or columns

### TRY FOR YOURSELF (UNGRADED EXERCISES)

3) Redo the join exercise above, using an inner join instead of a right join.  
Make sure the results make sense.

## `.reshape()`

Changes the object's shape

We illustrate creating pandas Series, extracting array of length 6, and reshaping to 3x2 array.

In [ ]:
# create a series
ser = pd.Series([1, 1, 2, 3, 5, 8])

# extract values
vals = ser.values

print('orig data:', vals)
print('orig type:', type(vals))
print('orig shape:', vals.shape)

# reshaping series
reshaped_vals = vals.reshape((3, 2))

print('\n reshaped vals:')
print(reshaped_vals)
print('\n new type:', type(reshaped_vals))
print('new shape:', reshaped_vals.shape)

Including -1 as one of the dimensions tells numpy: infer this dimension from the data and the other dimensions.

Example: enforce 3 columns:

In [ ]:
vals.reshape(-1,3)

Enforce 3 rows:

In [ ]:
vals.reshape(3,-1)

**IMPORTANT NOTE**  

Notice the shape of original array: `(6,)`  
This is a vector with one dimension, and is different from two-dimensional `(6,1)` array

### TRY FOR YOURSELF (UNGRADED EXERCISES)

4) Recreate the series from above with data [1, 1, 2, 3, 5, 8]  
Extract the data from the series and reshape to 2x3.  
Print both the reshaped data, and the dimensions.

## Categoricals

Categorical data takes discrete values where computation on the values does not make sense.  
Zip code is a typical example.

To include categoricals in models, they must be converted to numeric.  

### `get_dummies()`
Dummy code categorical data

The parameter `prefix` appends the prefix to column names (a good idea for later use)
<!-- - `drop_first`: remove first level, as only `k-1` variables needed to represent `k` levels
-->

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [ ]:
cats = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese']})

print('--categorical data')
print(cats)

cats = pd.get_dummies(cats.breed, prefix='breed')

print('\n')
print('--dummified categorical data')
print(cats)

Notice `burmese` was dropped (first level by alphabet)

### TRY FOR YOURSELF (UNGRADED EXERCISES)

5) The dataframe below contains two categoricals. Dummify each of them, giving them a prefix and dropping the first level from each.

Print the new dataframe to insure correctness.

Hint: You might want to dummify each column into separate new dataframes, and then merge them together by using:

`pd.concat([df1, df2], axis=1)`

In [ ]:
cats2 = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese'],
                      'color':['calico','white','seal point','cream','sable']})

---